#### Импортируем

In [1]:
from lxml import html
import requests
from datetime import datetime, timedelta
import re

#### Получаем страницу

In [2]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'}
response = requests.get('https://yandex.ru/news/', headers=header)
dom = html.fromstring(response.text)

#### Инициализируем структуру dom

In [3]:
news = dom.xpath("//div[contains(@class, 'mg-grid__col mg-grid__col')]/article[contains(@class, 'mg-card ')]")
news_list = []

#### Парсим страницу

In [4]:
today = datetime.now().strftime('%Y-%m-%d')
yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

for el in news:
    name = ''.join(el.xpath(".//h2/text()")).replace('\xa0', ' ')
    link = ''.join(el.xpath(".//a[@class='mg-card__link']/@href"))
    source_name = ''.join(el.xpath(".//a[@class='mg-card__source-link']/text()"))
    time = ''.join(el.xpath(".//span[@class='mg-card-source__time']/text()"))
    
    if 'вчера' in time.lower():
        time = ''.join(re.findall(r'\d\d:\d\d', time))
        date_time = f'{yesterday} {time}'
    else:
        date_time = f'{today} {time}'
        
    news_list.append({'name': name,
                     'link': link, 
                     'source_name': source_name,
                     'date_time': date_time})

#### Импортируем

In [5]:
from pymongo import MongoClient

#### Инициализируем БД

In [6]:
client = MongoClient('localhost', 27017)
db = client['news']
yandex = db.yandex

#### Добавляем собранные новости

In [7]:
yandex.insert_many(news_list)

#### Проверяем

In [9]:
list(yandex.find({}))[:5]

[{'_id': ObjectId('6117df128ef0f04e47b68fce'),
  'name': 'При крушении самолета Бе-200 Минобороны России в Турции погибли восемь человек',
  'link': 'https://yandex.ru/news/story/Prikrushenii_samoleta_Be-200_Minoborony_Rossii_vTurcii_pogibli_vosem_chelovek--1628ab9bee19e6c3abcb729347dc2305?lang=ru&rubric=index&fan=1&stid=3TFtGGWDlqqLk7BjT-DW&t=1628953829&tt=true&persistent_id=155770129',
  'source_name': 'ТАСС',
  'date_time': '2021-08-14 18:08'},
 {'_id': ObjectId('6117df128ef0f04e47b68fcf'),
  'name': 'Путин заявил о беспрецедентном характере природных бедствий в ряде регионов',
  'link': 'https://yandex.ru/news/story/Putin_zayavil_obesprecedentnom_kharaktere_prirodnykh_bedstvij_vryade_regionov--4727b2183dd5d0a1d4fd2c28f0b9833d?lang=ru&rubric=index&fan=1&stid=1N8C2-D1hmQYnLeac9A_&t=1628953829&tt=true&persistent_id=155768399',
  'source_name': 'RT на русском',
  'date_time': '2021-08-14 17:55'},
 {'_id': ObjectId('6117df128ef0f04e47b68fd0'),
  'name': 'В России третий день подряд фикс